In [233]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
import pandas as pd

Importing the Dataset

In [234]:
from sklearn.datasets import make_circles

In [235]:
make_circles

<function sklearn.datasets._samples_generator.make_circles(n_samples=100, *, shuffle=True, noise=None, random_state=None, factor=0.8)>

In [236]:
n_samples = 1000

Defining the input and output variable

In [237]:
X,y = make_circles(n_samples , noise = 0.03 , shuffle = True )

In [238]:
X

array([[-0.07683128,  0.82976691],
       [ 0.60411135, -0.79212936],
       [ 0.76748954,  0.67374773],
       ...,
       [ 0.7672709 ,  0.01289466],
       [-0.70858533, -0.19126901],
       [ 0.75184921,  0.30251286]])

In [239]:
y

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Preparing the DataFrame

In [240]:
circles_df = pd.DataFrame({"X1" : X[:,0] , "X2" : X[:,1] ,"label" : y})

In [241]:
circles_df.head(10)

,X1,X2,label
0,-0.076831,0.829767,1
1,0.604111,-0.792129,0
2,0.767490,0.673748,0
3,-0.237187,0.780564,1
4,-0.322448,0.915866,0
5,0.513240,-0.648223,1
6,-0.814669,0.622648,0
7,0.856162,0.592311,0
8,0.489827,-0.902810,0
9,0.782450,-0.166628,1


 Identifying the shape of the Dataset

In [242]:
print(f"The shape of input feature is {X.shape} , Shape of target feature is {y.shape}")

The shape of input feature is (1000, 2) , Shape of target feature is (1000,)


# 1 Turning our data to Tensors and creating the splits

In [243]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [244]:
X [:5] , y[:5]

(tensor([[-0.0768,  0.8298],
         [ 0.6041, -0.7921],
         [ 0.7675,  0.6737],
         [-0.2372,  0.7806],
         [-0.3224,  0.9159]]),
 tensor([1., 0., 0., 1., 0.]))

In [245]:
type(X) , X.dtype , type(y)

(torch.Tensor, torch.float32, torch.Tensor)

Splitting the data

In [246]:
from sklearn.model_selection import train_test_split
x_train , x_test = train_test_split(X , random_state = 42 , test_size = 0.2)
y_train , y_test = train_test_split(y , random_state = 42 , test_size = 0.2)

In [247]:
print(f"X_train's shape is {x_train.shape} , y_train's shape is {y_train.shape} , X_test's shape is {x_test.shape} , y_test's shape is {y_test.shape}")

X_train's shape is torch.Size([800, 2]) , y_train's shape is torch.Size([800]) , X_test's shape is torch.Size([200, 2]) , y_test's shape is torch.Size([200])


#2 Build or pick a pretrained model

To do so, we want to:
1. Setup device agonistic code so our code will run on an accelerator (GPU) if there is one


2. Construct a model (by subclassing nn.Module)


3. Define a loss function and optimizer


4. Create a training and test loop

In [248]:
#Make device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Now we've setup device agnostic code, let's create a model that:
1. Subclasses nn. Module (almost all models in PyTorch subclass nn. Module)
2. Create 2 nn.Linear() layers that are capable of handling the shapes of our data
3. Defines a forward() method that outlines the forward pass (or forward computation) of the model
4. Instatiate an instance of our model class and send it to the target device

In [249]:
class CircleModel(nn.Module):
  #1.Construct a model that subclasses nn.Module
  def __init__(self):
    super().__init__()

    #2.Create 2 nn.Linear layers capable of handling the shapes of our data
    self.layer_1 = nn.Linear(in_features = 2 , out_features = 5)
    self.layer_2 = nn.Linear(in_features = 5 , out_features = 1)

    #3.Define a forward() method that outlines the forward pass
  def forward(self,x):
    return self.layer_2(self.layer_1(x)) #x -> layer_1 -> layer_2 -> output

#4.Instantiate instance of our model and send it to the target device
model_0 = CircleModel()

In [250]:
model_0

CircleModel(
  (layer_1): Linear(in_features=2, out_features=5, bias=True)
  (layer_2): Linear(in_features=5, out_features=1, bias=True)
)

Random initialization of the model

In [251]:
list(model_0.parameters())

[Parameter containing:
 tensor([[ 0.5406,  0.5869],
         [-0.1657,  0.6496],
         [-0.1549,  0.1427],
         [-0.3443,  0.4153],
         [ 0.6233, -0.5188]], requires_grad=True),
 Parameter containing:
 tensor([0.6146, 0.1323, 0.5224, 0.0958, 0.3410], requires_grad=True),
 Parameter containing:
 tensor([[-0.0631,  0.3448,  0.0661, -0.2088,  0.1140]], requires_grad=True),
 Parameter containing:
 tensor([-0.2060], requires_grad=True)]

Replicating the above model using nn.Sequential()

This was the same as previous defined in class

In [252]:
model = nn.Sequential(
   nn.Linear(in_features = 2 , out_features = 5),
   nn.Linear(in_features = 5 , out_features = 1)
)
model

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): Linear(in_features=5, out_features=1, bias=True)
)

In [253]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.0829, -0.2872],
                      [ 0.4691, -0.5582],
                      [-0.3260, -0.1997],
                      [-0.4252,  0.0667],
                      [-0.6984,  0.6386]])),
             ('0.bias', tensor([-0.6007,  0.5459,  0.1177, -0.2296,  0.4370])),
             ('1.weight',
              tensor([[ 0.0697,  0.3613,  0.0489, -0.1410,  0.1202]])),
             ('1.bias', tensor([-0.1213]))])

Setting up Loss function and optimizer

In [254]:
loss_fn = nn.BCEWithLogitsLoss() #sigmoid activation function

In [255]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters() , learning_rate)

Acuuracy

In [256]:
from sklearn.metrics import accuracy_score

##3.Train Model

To train our model , we're going to need to build a training loop

1.Forward pass

2.Calculate the Loss

3.Optimizer zero grad

4.Loss backward (Backpropagation)

5.Optimizer step (gradient descent)

In [257]:
model

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): Linear(in_features=5, out_features=1, bias=True)
)

Our model outputs are going to be raw logits


We can convert these logits into prediction probabilities by passing them to some kind of activation function (eg. sigmoid for binary classification and softmax for multiclass classification)

Then we can convert our model's prediction probabilities to prediction labels by either rounding them or taking the argmax()

logits -> pred probs -> pred labels

In [258]:
model.eval()
with torch.inference_mode():
  y_logits = model(x_test)

In [259]:
y_logits[:5]

tensor([[ 0.0858],
        [ 0.0812],
        [-0.0671],
        [-0.0381],
        [ 0.1627]])

In [260]:
y_test[:5]

tensor([0., 0., 0., 1., 1.])

In [261]:
#Use the sigmoid function on our model logits to turn them into prediction probabilities
y_pred_probs = torch.sigmoid(y_logits)
y_pred_probs[:5]

tensor([[0.5214],
        [0.5203],
        [0.4832],
        [0.4905],
        [0.5406]])

Now it's time to change it to either 0 or 1

In [262]:
y_pred = torch.round(y_pred_probs)[:5]

In [263]:
#Removing the extra dimensions
y_pred.squeeze()

tensor([1., 1., 0., 0., 1.])

In [264]:
y_pred

tensor([[1.],
        [1.],
        [0.],
        [0.],
        [1.]])

Building a training and testing loop

In [265]:
x_train[:5]

tensor([[ 0.5198, -0.5714],
        [-0.9582,  0.0930],
        [ 0.3858,  0.9404],
        [-0.8026,  0.6636],
        [-0.1563, -0.8040]])

In [266]:
y_train[:5]

tensor([1., 0., 0., 0., 1.])

In [267]:
from sklearn.metrics import accuracy_score

In [268]:
torch.manual_seed(42)
epochs = 101
for epoch in range(epochs):
  model.train()
  with torch.inference_mode():
    #1.Forward pass
    train_logits = model(x_train).squeeze()
    train_pred = torch.round(torch.sigmoid(train_logits))

    #2.Calculate the loss/accuarcy
    loss_train = loss_fn(train_logits,y_train)
    accuracy_train = accuracy_score(train_pred,y_train)*100
    loss_train.requires_grad_()

     #3.Optimizer.zero_grad
    optimizer.zero_grad()

     #4.Backpropagation
    loss_train.backward()

    #5.Optimizer.step()
    optimizer.step()


  model.eval()
  with torch.inference_mode():

    #1.Forward pass
    test_logits = model(x_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))

    #2.Calculate the loss/accuracy
    loss_test = loss_fn(test_logits,y_test)
    accuracy_test = accuracy_score(test_pred,y_test)*100

    #3.Optimizer.zero_grad
    optimizer.zero_grad()
    loss_test.requires_grad_()

    #4.Backpropagation
    loss_test.backward()

    #5.Optimizer.step
    optimizer.step()


    if(epoch%20==0):
      print(f"The epoch's iteration number is {epoch} \n \n Loss of train is {loss_train} \n accuracy of train is {accuracy_train}")
      print(f"Loss of test is {loss_test} \n accuracy of test is {accuracy_test}")
      print("\n \n")


The epoch's iteration number is 0 
 
 Loss of train is 0.6982197761535645 
 accuracy of train is 53.0
Loss of test is 0.6931395530700684 
 accuracy of test is 54.50000000000001

 

The epoch's iteration number is 20 
 
 Loss of train is 0.6982197761535645 
 accuracy of train is 53.0
Loss of test is 0.6931395530700684 
 accuracy of test is 54.50000000000001

 

The epoch's iteration number is 40 
 
 Loss of train is 0.6982197761535645 
 accuracy of train is 53.0
Loss of test is 0.6931395530700684 
 accuracy of test is 54.50000000000001

 

The epoch's iteration number is 60 
 
 Loss of train is 0.6982197761535645 
 accuracy of train is 53.0
Loss of test is 0.6931395530700684 
 accuracy of test is 54.50000000000001

 

The epoch's iteration number is 80 
 
 Loss of train is 0.6982197761535645 
 accuracy of train is 53.0
Loss of test is 0.6931395530700684 
 accuracy of test is 54.50000000000001

 

The epoch's iteration number is 100 
 
 Loss of train is 0.6982197761535645 
 accuracy of t

#Improving a model

* Add more layers : Give the model more chances to learn about patterns in the data

* Add more hidden units : Go from 5 hidden units to 10 hidden units

* Changing the activation function

* Change the learning rate

* Change the loss function

Let's try and improve our model by :

* Adding more hidden units : 5 -> 10

* Increase the number of layers : 2 -> 3

* Increase the number of epochs : 100 -> 1000

In [269]:
class CircleModelV1(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features = 2 , out_features = 10)
    self.layer_2 = nn.Linear(in_features = 10 , out_features = 10)
    self.layer_3 = nn.Linear(in_features = 10 , out_features = 1)
  def forward(self,x):
    return self.layer_3(self.layer_2(self.layer_1(x)))

model_1 = CircleModelV1()
model_1

CircleModelV1(
  (layer_1): Linear(in_features=2, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=1, bias=True)
)

In [270]:
# Create a loss function
# Create an optimizer
# Write a training and evaluation loop for model_1

In [271]:
def accuracy(x,y):
  acc = torch.eq(x,y).sum().item()
  return acc/len(x)


In [272]:
from sklearn.metrics import accuracy_score

In [273]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params = model_1.parameters() , lr = 0.0001)

In [274]:
torch.manual_seed(42)
epochs = 1001

for epoch in range(epochs):

  #Training
  model_1.train()

  #1.Forward pass
  train_logits = model_1(x_train).squeeze()
  train_preds = torch.round(torch.sigmoid(train_logits))

  #2.Calculate loss/acc
  train_loss = loss_fn(train_preds,y_train)
  train_accuracy = accuracy_score(train_preds.detach().numpy() , y_train.detach().numpy())*100

  #3.Optimizer zero grad
  optimizer.zero_grad()

  #4.Loss Backward (Back Propagation)
  train_loss.backward()

  #5.Optimizer.step (gradoient descent)
  optimizer.step()


##Testing :
  model_1.eval()
  with torch.inference_mode():
    #1.Forward pass
    test_logits = model_1(x_test).squeeze()
    test_preds = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_preds,y_test)
    test_accuracy = accuracy_score(test_preds.detach().numpy() , y_test.detach().numpy())*100

  if(epoch % 250 == 0):
      print(f"The epoch's iteration number is {epoch} \n \n Loss of train is {train_loss} \n accuracy of train is {train_accuracy}")
      print(f"Loss of test is {test_loss} \n accuracy of test is {test_accuracy}")
      print("\n \n")

The epoch's iteration number is 0 
 
 Loss of train is 0.7613121271133423 
 accuracy of train is 50.125
Loss of test is 0.7509140968322754 
 accuracy of test is 52.0

 

The epoch's iteration number is 250 
 
 Loss of train is 0.7613121271133423 
 accuracy of train is 50.125
Loss of test is 0.7509140968322754 
 accuracy of test is 52.0

 

The epoch's iteration number is 500 
 
 Loss of train is 0.7613121271133423 
 accuracy of train is 50.125
Loss of test is 0.7509140968322754 
 accuracy of test is 52.0

 

The epoch's iteration number is 750 
 
 Loss of train is 0.7613121271133423 
 accuracy of train is 50.125
Loss of test is 0.7509140968322754 
 accuracy of test is 52.0

 

The epoch's iteration number is 1000 
 
 Loss of train is 0.7613121271133423 
 accuracy of train is 50.125
Loss of test is 0.7509140968322754 
 accuracy of test is 52.0

 

